In [1]:
!pip install -q ivy

In [2]:
import ivy
import torch
import tensorflow as tf
import numpy as np

2024-01-22 11:50:15.099601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-22 11:50:15.099643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-22 11:50:15.100960: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-22 11:50:15.108768: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-22 11:50:16.332076: W tensorflow/compiler/tf2

In [3]:
# create torch lstm layer
torch_lstm = torch.nn.LSTM(2, 2, 1).to("cuda")

# transpile lstm layer to tensorflow
x = torch.rand((5, 2, 2)).cuda()
tf_lstm = ivy.transpile(torch_lstm, source="torch", to="tensorflow", args=(x,))

/workspaces/tracer-transpiler/ivy_repo/ivy/utils/exceptions.py:383: UserWarning: The current backend: 'tensorflow' does not support inplace updates natively. Ivy would quietly create new arrays when using inplace updates with this backend, leading to memory overhead (same applies for views). If you want to control your memory management, consider doing ivy.set_inplace_mode('strict') which should raise an error whenever an inplace update is attempted with this backend.
  warnings.warn(
2024-01-22 11:50:26.221321: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14699 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0001:00:00.0, compute capability: 7.0
2024-01-22 11:50:26.625856: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902


In [4]:
# get output of original torch lstm layer
x = torch.rand((20, 32, 2)).cuda()
original_output = torch_lstm(x)

# get output of transpiled tf lstm layer with the same input
x = tf.constant(x.cpu().numpy())
transpiled_output = tf_lstm(x)

# verify the outputs are the same (with some tolerance)
np.allclose(original_output[0].detach().cpu(), transpiled_output[0].numpy(), atol=1e-7)

True

In [5]:
# run some benchmarks
from time import perf_counter

x = torch.rand((20, 32, 2)).cuda()
N_RUNS = 1000

# time the original torch lstm
s = perf_counter()
for _ in range(N_RUNS):
  torch_lstm(x)
original_torch_time = perf_counter() - s

# compile transpiled tf lstm
x = tf.constant(x.cpu().numpy())
tf_lstm = tf.autograph.experimental.do_not_convert(tf_lstm)
compiled_tf_lstm = tf.function(tf_lstm)
compiled_tf_lstm(x)

# time the transpiled tf lstm
s = perf_counter()
for _ in range(N_RUNS):
  compiled_tf_lstm(x)
transpiled_tf_time = perf_counter() - s

# time tf's own lstm layer (also compiled) for comparison
original_tf_lstm = tf.keras.layers.LSTM(2, time_major=True, return_sequences=True)
original_tf_lstm = tf.function(original_tf_lstm)
original_tf_lstm(x)

s = perf_counter()
for _ in range(N_RUNS):
  original_tf_lstm(x)
original_tf_time = perf_counter() - s

# as we can see, the transpiled tf lstm has comparable performance to tf's own lstm layer
print(f'transpiled tf time is {transpiled_tf_time / original_torch_time}x slower than torch\'s lstm')
print(f'original tf lstm time is {original_tf_time / original_torch_time}x slower than torch\'s lstm')

2024-01-22 11:50:30.698440: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


transpiled tf time is 3.725307350738295x slower than torch's lstm
original tf lstm time is 4.440824652724787x slower than torch's lstm
